In [19]:
import scipy.io
import numpy as np

data = scipy.io.loadmat('Flujo2019.mat')

# ver las variables
print(data.keys())


dict_keys(['__header__', '__version__', '__globals__', 'None', 'FlujoAgosto', 'FlujoDiciembre', 'FlujoJulio', 'FlujoJunio', 'FlujoNoviembre', 'FlujoOctubre', 'FlujoSeptiembre', '__function_workspace__'])


In [23]:
for key in data.keys():
    if not key.startswith('__'):  
        print(f"{key}: {data[key].shape}")


None: (1,)
FlujoAgosto: (1, 3965)
FlujoDiciembre: (1, 3867)
FlujoJulio: (1, 3969)
FlujoJunio: (1, 1459)
FlujoNoviembre: (1, 3775)
FlujoOctubre: (1, 2363)
FlujoSeptiembre: (1, 2362)


In [22]:
flujo_julio = data['FlujoJulio'].flatten()
flujo_agosto = data['FlujoAgosto'].flatten()
flujo_septiembre = data['FlujoSeptiembre'].flatten()
flujo_octubre = data['FlujoOctubre'].flatten()
flujo_noviembre = data['FlujoNoviembre'].flatten()
flujo_diciembre = data['FlujoDiciembre'].flatten()

In [24]:
# normalizar los datos para la generación de la red 
min_length = min(len(flujo_julio), len(flujo_agosto), len(flujo_septiembre),
                 len(flujo_octubre), len(flujo_noviembre), len(flujo_diciembre))

flujo_julio = flujo_julio[:min_length]
flujo_agosto = flujo_agosto[:min_length]
flujo_septiembre = flujo_septiembre[:min_length]
flujo_octubre = flujo_octubre[:min_length]
flujo_noviembre = flujo_noviembre[:min_length]
flujo_diciembre = flujo_diciembre[:min_length]


Se considera una entrada $x$, una salida $y$

In [25]:
X = np.column_stack((flujo_julio, flujo_agosto, flujo_septiembre, flujo_octubre, flujo_noviembre))
Y = flujo_diciembre 



In [26]:
X = (X - np.min(X)) / (np.max(X) - np.min(X))
Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y))


La función sigmoide logística se define como

$ \sigma(z) = \frac{1}{1 + e^{-z}} $

La derivada es

$ \sigma'(z) = \sigma(z) \cdot (1 - \sigma(z)) $

In [32]:
print("Valores NaN en Y:", np.isnan(Y).sum())


Valores NaN en Y: 2362


In [33]:
# Encontrar los índices donde Y es válido (no NaN)
valid_indices = ~np.isnan(Y)

# Filtrar X y Y para eliminar filas con NaN en Y
X = X[valid_indices]
Y = Y[valid_indices]


In [34]:
print("Valores NaN en Y:", np.isnan(Y).sum())

Valores NaN en Y: 0


In [31]:
import numpy as np
import scipy.io

# 🔹 1. Cargar los datos desde el archivo .mat
data = scipy.io.loadmat('Flujo2019.mat')

# 🔹 2. Extraer y convertir en vectores 1D
flujo_julio = data['FlujoJulio'].flatten()
flujo_agosto = data['FlujoAgosto'].flatten()
flujo_septiembre = data['FlujoSeptiembre'].flatten()
flujo_octubre = data['FlujoOctubre'].flatten()
flujo_noviembre = data['FlujoNoviembre'].flatten()
flujo_diciembre = data['FlujoDiciembre'].flatten()

# 🔹 3. Alinear los datos (cortar al menor tamaño disponible)
min_length = min(len(flujo_julio), len(flujo_agosto), len(flujo_septiembre),
                 len(flujo_octubre), len(flujo_noviembre), len(flujo_diciembre))

flujo_julio = flujo_julio[:min_length]
flujo_agosto = flujo_agosto[:min_length]
flujo_septiembre = flujo_septiembre[:min_length]
flujo_octubre = flujo_octubre[:min_length]
flujo_noviembre = flujo_noviembre[:min_length]
flujo_diciembre = flujo_diciembre[:min_length]

# 🔹 4. Definir X (entrada) y Y (salida esperada)
X = np.column_stack((flujo_julio, flujo_agosto, flujo_septiembre, flujo_octubre, flujo_noviembre))
Y = flujo_diciembre  # Queremos predecir diciembre basado en meses anteriores

# 🔹 5. Normalizar los datos
X_min, X_max = np.min(X), np.max(X)
Y_min, Y_max = np.min(Y), np.max(Y)

X = (X - X_min) / (X_max - X_min)
Y = (Y - Y_min) / (Y_max - Y_min)

print("Min X:", X_min, "Max X:", X_max)
print("Min Y:", Y_min, "Max Y:", Y_max)

X = (X - np.min(X)) / (np.max(X) - np.min(X))
Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y))

# ---------------------------------------
# 🔹 6. Inicializar pesos y sesgos (VARIABLES GLOBALES)
w1, w2, w3, w4, w5 = np.random.uniform(-0.5, 0.5, 5)
b2, b4, b5, b_out = np.random.uniform(-0.5, 0.5, 4)
w_out1, w_out2 = np.random.randn(), np.random.randn()  # Pesos de salida
alpha = 0.05  # Tasa de aprendizaje

# 🔹 7. Función de activación (sigmoide) y su derivada
def sigmoid(z):
    z = np.clip(z, -500, 500)  # Evita valores extremos
    return 1 / (1 + np.exp(-z))


def sigmoid_derivative(a):
    return a * (1 - a)  # Usa la salida de sigmoide directamente

# ---------------------------------------
# 🔹 8. Forward propagation
def forward(x):
    global w1, w2, w3, w4, w5, b2, b4, b5, b_out, w_out1, w_out2

    z2 = np.dot(x, w2) + b2
    a2 = sigmoid(z2)

    z4 = np.dot(x, w4) + b4
    a4 = sigmoid(z4)

    z5 = np.dot(x, w5) + b5
    a5 = sigmoid(z5)

    y_hat = w_out1 * a2 + w_out2 * a4 + b_out
    return y_hat, a2, a4, a5, z2, z4, z5

# 🔹 9. Función de pérdida
def loss(y_real, y_pred):
    return 0.5 * (y_real - y_pred) ** 2

# ---------------------------------------
# 🔹 10. Backpropagation (ajuste de pesos y sesgos)
def backward(x, y_real, y_pred, a2, a4, a5, z2, z4, z5):
    global w1, w2, w3, w4, w5, b2, b4, b5, b_out, w_out1, w_out2, alpha

    error = y_pred - y_real

    # Gradientes de la capa de salida
    d_w_out1 = error * a2
    d_w_out2 = error * a4
    d_b_out = error

    # Gradientes de la capa oculta
    d_a2 = error * w_out1
    d_a4 = error * w_out2

    d_z2 = d_a2 * sigmoid_derivative(z2)
    d_z4 = d_a4 * sigmoid_derivative(z4)

    d_w2 = np.dot(x.T, d_z2)
    d_w4 = np.dot(x.T, d_z4)

    d_b2 = np.sum(d_z2)
    d_b4 = np.sum(d_z4)

    # 🔹 11. Actualización de pesos y sesgos
    w_out1 -= alpha * d_w_out1
    w_out2 -= alpha * d_w_out2
    b_out -= alpha * d_b_out

    w2 -= alpha * d_w2
    w4 -= alpha * d_w4
    b2 -= alpha * d_b2
    b4 -= alpha * d_b4

# ---------------------------------------
# 🔹 12. Función de entrenamiento
def train(X, Y, epochs=1000):
    for epoch in range(epochs):
        total_loss = 0
        for x, y_real in zip(X, Y):
            y_pred, a2, a4, a5, z2, z4, z5 = forward(x)
            total_loss += loss(y_real, y_pred)
            backward(x, y_real, y_pred, a2, a4, a5, z2, z4, z5)

        if epoch % 100 == 0:
            print(f"Época {epoch}, Pérdida: {total_loss / len(X)}")

# ---------------------------------------
# 🔹 13. ENTRENAR LA RED
train(X, Y, epochs=1000)

# 🔹 14. HACER UNA PREDICCIÓN
test_value = np.array([[0.3, 0.4, 0.5, 0.6, 0.7]])  # Valores normalizados
prediction, _, _, _, _, _, _ = forward(test_value)
print(f"Predicción para flujo de diciembre: {prediction}")


Min X: 0.0 Max X: 6.74
Min Y: nan Max Y: nan
Época 0, Pérdida: [nan nan nan nan nan]
Época 100, Pérdida: [nan nan nan nan nan]
Época 200, Pérdida: [nan nan nan nan nan]
Época 300, Pérdida: [nan nan nan nan nan]
Época 400, Pérdida: [nan nan nan nan nan]


KeyboardInterrupt: 

In [28]:
X_min, X_max = np.min(X), np.max(X)
Y_min, Y_max = np.min(Y), np.max(Y)

X = (X - X_min) / (X_max - X_min)
Y = (Y - Y_min) / (Y_max - Y_min)

print("Min X:", X_min, "Max X:", X_max)
print("Min Y:", Y_min, "Max Y:", Y_max)


Min X: 0.0 Max X: 1.0
Min Y: nan Max Y: nan


Variables globales como pesos y sesgos